In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**importing necessary modules**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

**importing the dataset**

In [ ]:
train = pd.read_csv('../input/titanicdataset-traincsv/train.csv')
train.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
sns.heatmap(train.isnull(),xticklabels=True, yticklabels=False, cbar=False, cmap='viridis')

It seems that, Age and Cabin columns has none values. Roughly 20% Age data missing, which is reasonable replacable. 
Looking at the Cabin column, it looks we are just missing too much data, probabaly will drp this column.

## EDA

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived', data=train)

In [ ]:
train['Survived'].value_counts()

## from obove plot, it seems this is balanced data set

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train)

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train)

In [ ]:
sns.countplot(x='SibSp',data=train)

In [ ]:
sns.pairplot(train, hue='Survived')

In [ ]:
sns.distplot(train['Age'])

In [ ]:
sns.distplot(train['Fare'])

## From above code, it assumes Age column follows Guassion / Normal Distribution and  the Fare columns is skewed.

## Feature Engineering

<h3>Imputing the null values with mean<h3>

In [ ]:
sns.boxplot(x='Pclass',y='Age',data=train)

## from above plot, average mean age for 1st class passenger is 37, for 2nd class is 30, for 3rd class is 25

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    if pd.isnull(Age):
        if Pclass == 1:
            return 37
        elif Pclass == 2:
            return 29
        else:
            return 24
    else:
        return Age


In [ ]:
train['Age'] = train[['Age','Pclass']].apply(impute_age, axis=1)

In [ ]:
train.isnull().sum()

In [ ]:
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
train.drop('Cabin',axis=1,inplace= True)
train.head()

In [ ]:
train.isnull().sum()

<h3>We have removed the nan values<h3>

## Lets handle the outlier

In [ ]:
train['Age'].describe()

In [ ]:
fig = train.Age.hist(bins=50)
fig.set_title('Age')
fig.set_xlabel('Age')
fig.set_ylabel('No of passenger')

In [ ]:
## as Age column follows Guassion Distribution, we will calculate the boundries which differntiate the outliers.
upper_boundry = train['Age'].mean() + 3 * train['Age'].std()
lower_boundry = train['Age'].mean() - 3 * train['Age'].std()
print(lower_boundry)
print(upper_boundry)
print(train['Age'].mean())

In [ ]:
## Fare column is skewed
fig = train.Fare.hist(bins=50)
fig.set_title('Fare')
fig.set_xlabel('Fare')
fig.set_ylabel('No of passenger')

In [ ]:
train.boxplot(column='Fare')

In [ ]:
train['Fare'].describe()

In [ ]:
## lets compute the inter quantile range to calculate the boundries
IQR=train.Fare.quantile(0.75)-train.Fare.quantile(0.25)
IQR

In [ ]:
lower_bridge = train['Fare'].quantile(0.25)-(IQR*1.5)
upper_bridge = train['Fare'].quantile(0.75)+(IQR*1.5)
print(lower_bridge)
print(upper_bridge)

In [ ]:
## for extreme outlier
lower_bridge = train['Fare'].quantile(0.25)-(IQR*3)
upper_bridge = train['Fare'].quantile(0.75)+(IQR*3)
print(lower_bridge)
print(upper_bridge)

In [ ]:
train=train.copy()

In [ ]:
train.loc[train['Age']>=68.80,'Age']=68.80

In [ ]:
train.loc[train['Fare']>=100.2688,'Fare']=100.2688

In [ ]:
fig = train.Fare.hist(bins=50)
fig.set_title('Fare')
fig.set_xlabel('Fare')
fig.set_ylabel('No of passenger')

In [ ]:
fig = train.Age.hist(bins=50)
fig.set_title('Age')
fig.set_xlabel('Age')
fig.set_ylabel('No of passenger')

In [ ]:
train.head()

## Lets transform the other columns to numearical value 

In [ ]:
pd.get_dummies(train['Embarked'],drop_first=True).head()

In [ ]:
sex = pd.get_dummies(train['Sex'],drop_first = True)
embark = pd.get_dummies(train['Embarked'],drop_first = True)

In [ ]:
train.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
train.head()

In [ ]:
train = pd.concat([train,sex,embark],axis=1)

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
X = train.drop(['Survived'],axis=1)
y = train['Survived']
bestfeatures = SelectKBest(chi2, k=8)
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Attributes','Score']

In [ ]:
print(featureScores.nlargest(9,'Score'))

In [ ]:
train.shape

## Scaling

In [ ]:
## train test split
from sklearn.model_selection import train_test_split
X = train.drop(['Survived'],axis=1)
y = train['Survived']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train.values)

In [ ]:
X_train = scaler.transform(X_train)
X_train

In [ ]:
X_test = scaler.transform(X_test)
X_test

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score, accuracy_score

## Model Creation

**Building Logistics Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
log_cls = LogisticRegression()
log_cls.fit(X_train,y_train)
ytrain_pred = log_cls.predict_proba(X_train)
print('RF train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = log_cls.predict(X_test)
ytest_pred = log_cls.predict_proba(X_test)
print('RF test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

In [ ]:
ytrain_pred

## Will apply the Genetic algorithm and will check the accuracy.

In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start=10, stop=1000, num=10)]
max_features = ['auto','sqrt','log2']
max_depth = [int(x) for x in np.linspace(10,1000,10)]
min_samples_split = [2,5,10,14]
min_samples_leaf = [1,3,5,8]
param = {'n_estimators':n_estimators, 'max_features':max_features, 'max_depth':max_depth, 'min_samples_split':min_samples_split,
        'min_samples_leaf':min_samples_leaf, 'criterion':['entropy','gini']}
print(param)

In [ ]:
from tpot import TPOTClassifier
tpot_classifier = TPOTClassifier(generations=5,population_size=24,offspring_size=12,verbosity=2,early_stop=12,
                                config_dict={'sklearn.ensemble.RandomForestClassifier': param},cv=4, scoring='accuracy')
tpot_classifier.fit(X_train,y_train)

In [ ]:
accuracy = tpot_classifier.score(X_test,y_test)
print(accuracy)

## Now will check the accuracy with Optuna

In [ ]:
import optuna
import sklearn
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators',100,2000,10)
    max_depth = trial.suggest_float('max_depth',10,1000)
    cls = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
    return sklearn.model_selection.cross_val_score(cls,X_train,y_train,n_jobs=-1,cv=4).mean()


In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective,n_trials=100)
trial = study.best_trial

print('Accuracy: {}'.format(trial.values))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
trial

In [ ]:
study.best_params

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=1150, max_depth=681.0458391285997)
rf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
y_pred=rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

**Building Decesion tree Classifier**

In [ ]:
from sklearn import tree
dt_cls = tree.DecisionTreeClassifier()
dt_cls.fit(X_train, y_train) 
ytrain_pred = dt_cls.predict_proba(X_train)
print('DT train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = dt_cls.predict(X_test)
ytest_pred = dt_cls.predict_proba(X_test)
print('DT test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

**Building Random Forest Classifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_cls = RandomForestClassifier()
rf_cls.fit(X_train,y_train)
ytrain_pred = rf_cls.predict_proba(X_train)
print('RF train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = rf_cls.predict(X_test)
ytest_pred = rf_cls.predict_proba(X_test)
print('RF test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

**Building Gradient Boosting Classifier**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_cls = GradientBoostingClassifier()
gb_cls.fit(X_train,y_train)
ytrain_pred = gb_cls.predict_proba(X_train)
print('GB train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = gb_cls.predict(X_test)
ytest_pred = gb_cls.predict_proba(X_test)
print('GB test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

## Ada Boost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_cls = AdaBoostClassifier()
ada_cls.fit(X_train,y_train)
ytrain_pred = ada_cls.predict_proba(X_train)
print('GB train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = ada_cls.predict(X_test)
ytest_pred = ada_cls.predict_proba(X_test)
print('GB test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

## KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN_cls = KNeighborsClassifier()
KNN_cls.fit(X_train,y_train)
ytrain_pred = KNN_cls.predict_proba(X_train)
print('KNN train roc.auc: {}'.format(roc_auc_score(y_train,ytrain_pred[:,1])))
y_pred = KNN_cls.predict(X_test)
ytest_pred = KNN_cls.predict_proba(X_test)
print('KNN test roc.auc: {}'.format(roc_auc_score(y_test,ytest_pred[:,1])))
print('Accuracy score: {}'.format(accuracy_score(y_test,y_pred)))

## Now we will focus on selecting the best threshold for max accuracy

In [ ]:
pred = []
for model in [log_cls,dt_cls,rf_cls,gb_cls,ada_cls,KNN_cls]:
    pred.append(pd.Series(model.predict_proba(X_test)[:,1]))
final_prediction = pd.concat(pred,axis=1).mean(axis=1)
print('Ensemble test roc.auc: {}',format(roc_auc_score(y_test,final_prediction)))

In [ ]:
pd.concat(pred,axis=1)

In [ ]:
final_prediction

In [ ]:
### Calculate the roc curve
fpr, tpr, thresholds = roc_curve(y_test,final_prediction)
thresholds

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = []
for thres in thresholds:
    y_pred = np.where(final_prediction>thres,1,0)
    accuracy.append(accuracy_score(y_test,y_pred, normalize=True))
    
accuracy = pd.concat([pd.Series(thresholds), pd.Series(accuracy)],axis=1)
accuracy.columns = ['thresholds', 'accuracy']
accuracy

In [ ]:
accuracy.sort_values(by='accuracy',ascending=False, inplace=True)
accuracy.head()

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0,1],[0,1], color='darkblue')
    plt.xlabel('False Positive rate')
    plt.ylabel('True Positive rate')
    plt.title('Receiver Opearting Char Curve')
    plt.legend()
    plt.show()

In [ ]:
plot_roc_curve(fpr, tpr)